In [1]:
import sys
import os
# Add the parent directory of the notebook (cxb1114) to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)  

import torch
import pytorch_lightning.callbacks.model_checkpoint
import pytorch_lightning.callbacks.early_stopping
torch.serialization.add_safe_globals([
    pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint,
    pytorch_lightning.callbacks.early_stopping.EarlyStopping
])
import torchaudio

import IPython.display as ipd

## My test data

In [3]:
# def multi_decoder_separate_and_play(file_path, model):
#     # Load audio
#     waveform, sample_rate = torchaudio.load(file_path)
    
#     # Convert to mono if necessary
#     if waveform.shape[0] > 1:
#         waveform = waveform.mean(dim=0, keepdim=True)
    
#     # Resample if needed
#     target_sr = 8000  # or whatever your model expects
#     if sample_rate != target_sr:
#         resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sr)
#         waveform = resampler(waveform)
#         sample_rate = target_sr

#     # Add batch dimension
#     waveform = waveform.unsqueeze(0)  # shape: (1, 1, samples)

#     # est_sources shape: (batch, channels, samples, num_sources) or similar
#     # Adjust indexing as needed for your model's output
#     if hasattr(est_sources, "shape") and len(est_sources.shape) == 4:
#         est_sources = est_sources[0]  # remove batch dimension

#     num_sources = est_sources.shape[0] if len(est_sources.shape) == 3 else est_sources.shape[1]
#     print(num_sources)
#     base_name = os.path.splitext(os.path.basename(file_path))[0]

#     # Save separated sources and collect file paths
#     source_paths = []
#     for i in range(num_sources):
#         # Adjust indexing depending on est_sources shape
#         source = est_sources[i] if len(est_sources.shape) == 3 else est_sources[:, i, :]
#         out_path = f"{base_name}_multi_source{i+1}.wav"
#         torchaudio.save(out_path, source.detach().cpu(), sample_rate)
#         source_paths.append(out_path)

#     # Display original and separated audios
#     audios = [ipd.Audio(filename=file_path)]
#     audios += [ipd.Audio(filename=src) for src in source_paths]

#     for audio in audios:
#         ipd.display(audio)

In [5]:
file_path = r"C:\Users\conno\Desktop\UoB_Project\cxb1114\Multi_Decoder_DPRNN\2_mixture.wav"
mixture, sample_rate = torchaudio.load(file_path)
print("mixture audio(Input)")
display(ipd.Audio(data=mixture, rate=sample_rate))

mixture audio(Input)


what is n_src of below? Can it be changed? Or is this the complete model that detects n_src?

In [6]:
model = MultiDecoderDPRNN.from_pretrained("JunzheJosephZhu/MultiDecoderDPRNN").eval()
if torch.cuda.is_available():
    model.cuda()
    mixture = mixture.cuda()
    
sources_est = model.separate(mixture).cpu()
for i, source in enumerate(sources_est):
  print(f"estimated source {i}")
  display(ipd.Audio(data=source, rate=sample_rate))

estimated source 0


estimated source 1


In [8]:
file_path = r"C:\Users\conno\Desktop\UoB_Project\cxb1114\data\libricss\OV40\overlap_ratio_40.0_sil0.1_1.0_session0_actual39.5\record\utterances\utterance_0.wav"

# Load audio
mixture, sample_rate = torchaudio.load(file_path)


# If model expects mono, convert
if mixture.shape[0] > 1:
    mixture = mixture.mean(dim=0, keepdim=True)


# # If model expects a specific sample rate, resample
# target_sr = 8000 
# if sample_rate != target_sr:
#     resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sr)
#     mixture = resampler(mixture)
#     sample_rate = target_sr
    
# Play processed audio
print("mixture audio (Input)")
display(ipd.Audio(data=mixture, rate=sample_rate))

# Move to GPU if available
if torch.cuda.is_available():
    model.cuda()
    mixture = mixture.cuda()


sources_est = model.separate(mixture).cpu()
for i, source in enumerate(sources_est):
    print(f"estimated source {i}")
    display(ipd.Audio(data=source, rate=sample_rate))

mixture audio (Input)


estimated source 0


estimated source 1
